# Systematics impact: Grouped Impact Table method

*Michele Pinamonti - INFN Sezione di Trieste*

This part shows an example of a way to quantify the impact of systematic uncertainties, by splitting the statisical component and a number of systematic categories. The method used is the so-called Grouped Impact Table. It consists in fixing a number of nuisance parameters to their post-fit values and perform fits with each group fixed, and then extracting the impact of each group by comparing the resulting total uncertainty on the parameter of interest with the one from the nominal fit.




In [ ]:
using namespace RooStats;

Open file containing workspace and extract it

In [ ]:
TFile *f = new TFile("../ws/ATLASIT_prova_combined_ATLASIT_prova_model.root");
RooWorkspace *w = (RooWorkspace*)f->Get("combined");

Get observed data from the workspace

In [ ]:
RooDataSet *dataset = (RooDataSet*)w->data("obsData");

Perform the nominal S+B fit to data (but forcing RooFit and Minuit to stay silent first).
This time we switch Minos ON, so that we get asymmetric error bars on the result.

In [ ]:
RooMsgService::instance().setGlobalKillBelow(RooFit::FATAL);
w->var("mu_ttH")->setVal(0);
w->var("mu_ttH")->setConstant(false);
w->pdf("simPdf")->fitTo(*dataset,RooFit::PrintLevel(-1));

Get the ModelConfig, the POI and save the POI fitted value and uncertainty

In [ ]:
ModelConfig *mc = (ModelConfig*)w->obj("ModelConfig");
RooRealVar *poi = (RooRealVar*)mc->GetParametersOfInterest()->first();
double mu_hat = poi->getVal();
double mu_hat_err_up = poi->getErrorHi();
double mu_hat_err_down = poi->getErrorLo();

Print nominal fit result

In [ ]:
cout << "Nominal fit:" << endl;
cout << "  POI =";
cout << Form(" %+.3f",mu_hat);
cout << Form(" %+.3f",mu_hat_err_up);
cout << " / ";
cout << Form(" %+.3f",mu_hat_err_down);
cout << endl;

Save a "snapshot", containing parameter values after nominal fit (to be loaded afeterward)

In [ ]:
w->saveSnapshot("nominal_snapshot", *mc->GetPdf()->getParameters(dataset));

Now the main part:
- define groups of systematics as a set of corresponding nuisance parameter (NP) names
- for each group, fix the corresponding NPs and re-do the fit
- extract the impact of this group as the difference in quadrature between the obtained total error on POI and the nominal error on POI
- the statistical uncertainty, is then quantified by fixing all the NPs

In [ ]:
map<string,vector<string>> systGroups;
systGroups["Lumi"] = {"alpha_lumi"};
systGroups["JES"] = {"alpha_JES_Scenario1_NP1"};
systGroups["BkgXS"] = {"alpha_stXsec","alpha_ttXsec"};

map<string,float> impactMapUp;
map<string,float> impactMapDown;

// For stat-uncertainty, fix all the NPs
for(auto np_tmp : *mc->GetNuisanceParameters()){
    RooRealVar* np = (RooRealVar*)np_tmp;
    
    string np_name = np->GetName();
    
    // exclude parameters that are not "alpha_" (to exclude e.g. normalization factors and keep only NPs associated with systematics)
    if(np_name.find("alpha_")==string::npos) continue;
    
    np->setConstant(true);
}
w->pdf("simPdf")->fitTo(*dataset,RooFit::PrintLevel(-1));
impactMapUp["Statistics"] = poi->getErrorHi();
impactMapDown["Statistics"] = poi->getErrorLo();

// Then loop over the systematic groups, and fix each of the groups
for(const auto &group_pair : systGroups){
    
    string group_name = group_pair.first;
    auto group = group_pair.second;
    // remember to reload the snapshot!
    w->loadSnapshot("nominal_snapshot");
    for(auto np_name : group){
        RooRealVar* np = w->var(np_name.c_str());
        np->setConstant(true);
    }
    w->pdf("simPdf")->fitTo(*dataset,RooFit::PrintLevel(-1));
    double mu_hat_err_up_tmp = poi->getErrorHi();
    double mu_hat_err_down_tmp = poi->getErrorLo();
    impactMapUp[group_name] = sqrt(mu_hat_err_up*mu_hat_err_up - mu_hat_err_up_tmp*mu_hat_err_up_tmp);
    impactMapDown[group_name] = -sqrt(mu_hat_err_down*mu_hat_err_down - mu_hat_err_down_tmp*mu_hat_err_down_tmp);
}

Print the resulting table

In [ ]:
cout << "---------------------------------" << endl;
for(auto tmp_pair : impactMapUp){
    string group_name = tmp_pair.first;
    cout << setw(10);
    cout << group_name;
    cout << "\t";
    cout << Form("%+.3f",impactMapUp[group_name]);
    cout << " / ";
    cout << Form("%+.3f",impactMapDown[group_name]);    
    cout << endl;
}

Homework: compare with the outptus from the "ranking plot" method